In [10]:
import pandas as pd
import csv
import os, glob
import re

In [11]:
def download_permits_data(year, month):
    """
    Function that will download monthly permits data (in .txt format) from the census bureau
    
    Inputs -
    year - str
        the year of data you want to download (2014 - 2019)
    month - str
        the month of data you want to download (01, 02, ..., 12)
        
    Output - 
    downloads txt file of data for the specified month and year to the working directory
    """
    
    # month and year must be strings
    assert type(year)==str and type(month)==str, \
        "year and month parameters should be strings"
    
    #check that month and year are in the correct format
    url = year+month
    check = re.compile("^[2][0][1][4-9][0-1][0-9]$") 
    assert len(check.findall(url))==1, \
        """year is confined to the years '2014' to '2019' and month is confined to '01' to '12' \n 
        (remember to put a zero before the month if it is before October)"""
    
    #download data
    if f"{year}-{month}.txt" in os.listdir(): 
        os.remove(f"{year}-{month}.txt")
    os.system(f"curl https://www.census.gov/construction/bps/txt/tb3u{url}.txt >> {year}-{month}.txt")

In [12]:
def preprocess_txt(path_to_txt_file):
    """
    remove newline characters and spaces to allow txt files to be space delimmited
    
    Inputs -
    path to unprocessed txt file directly downloaded from the census
    
    Output -
    new txt representing the raw data without the superfluous newline and whitespace characters
    """
    f_in = open(path_to_txt_file,'r')
    txt = f_in.read()
    
    #remove newline characters after metro names
    txt = txt.replace(',\n',',') \
             .replace(', ',',') \
             .replace(', ',',') \
             .replace('\n ','')
    
    #find and remove whitespace characters between non-decimals
    non_decimal = re.compile('[a-zA-Z]\s[a-zA-Z]')
    txt = non_decimal.sub('-', txt)
    
    output_filename = f"{path_to_txt_file[:-4]}_processed.txt"
    if output_filename in os.listdir():
        os.remove(output_filename)
        
    f_out = open(output_filename,'w')
    f_out.write(txt)
    
    f_out.close()
    f_in.close()
    

In [13]:
def txt_to_csv(path_to_txt_file):
    """
    create csv through space delimitted txt file
    
    Input -
    processed txt of monthly permit data
    Output -
    csv of monthly permit data
    """
    csv_filename = f"{path_to_txt_file.split('/')[-1][:-4]}.csv"
    with open(path_to_txt_file.split('/')[-1]) as fin, \
        open(csv_filename, 'w') as fout:
        
        o=csv.writer(fout)
        for line in fin:
            o.writerow(line.split())
    
    #the text processing ruins the columns names and so we'll add them back in
    data = pd.read_csv(csv_filename)
    #percent = data.With
    data = data.iloc[1:len(data)-1]
    data = data.dropna(axis=1)
    
    #data['Monthly Coverage Percent'] = percent
    print(data)
    data.columns = ['CSA','CBSA','Name','Total','1 Unit','2 Units','3 & 4 Units',
                     '5 or more','Number of structures with 5 units or more',
                     'Monthly Coverage Percent Percent']
    
    data.to_csv(csv_filename, index=False)

In [14]:
def add_month_year_data(csv_filename):
    """
    adds month and year data to the monthly permits data
    
    Input-
    csv: str
        filename of csv in working directory
    output-
    input csv with temporal data added in columns
    """
    #import data into pandas
    csv_pd = pd.read_csv(csv_filename)
    
    #parse temportal data from csv filename (assumes file name is of the format YYYY-MM_processed)
    month = (csv_filename
            .split('-')[-1]
            .split('_')[0]
            )
    year  = csv_filename.split('-')[0]
    
    csv_pd['month'] = month
    csv_pd['year']  = year
    
    csv_pd.to_csv(csv_filename)
    

In [15]:
population = pd.read_csv('cbsa-population.csv', encoding = "ISO-8859-1")

#filter down dataset
population = (population[population.LSAD == 'Metropolitan Statistical Area'][['NAME']+[col for col in population.columns if 'POPESTIMATE' in col]] # or 'NETMIG' in col]]
              #.reset_index()
             )

#rename columns to years
population.rename(columns={col:col[-4:] for col in population.columns},
                  inplace=True
                 )

#unpivot population table
population = (population
              .set_index('NAME')
              .unstack()
              .reset_index(name='population_est')
              .rename(columns={'level_0':'year', 'NAME':'Name'})
             )

population.year = population.year.astype(int)
population.population_est = population.population_est.astype(int)
population.Name = population.Name.apply(lambda x: x.replace('-',' '))
population.Name = population.Name.apply(lambda x: x.replace('St. ','St.'))

In [16]:
population

,year,Name,population_est
0,2010,"Abilene, TX",165585
1,2010,"Akron, OH",703031
2,2010,"Albany, GA",154145
3,2010,"Albany Lebanon, OR",116891
4,2010,"Albany Schenectady Troy, NY",871082
...,...,...,...
3835,2019,"Yakima, WA",250873
3836,2019,"York Hanover, PA",449058
3837,2019,"Youngstown Warren Boardman, OH PA",536081
3838,2019,"Yuba City, CA",175639


In [17]:
netmig = pd.read_csv('cbsa-population.csv', encoding = "ISO-8859-1")

#filter down dataset
netmig = (netmig[netmig.LSAD == 'Metropolitan Statistical Area'][['NAME']+[col for col in netmig.columns if 'NETMIG' in col]]
              #.reset_index()
             )

#rename columns to years
netmig.rename(columns={col:col[-4:] for col in netmig.columns},
              inplace=True
             )

#unpivot population table
netmig = (netmig
          .set_index('NAME')
          .unstack()
          .reset_index(name='net_migration')
          .rename(columns={'level_0':'year', 'NAME':'Name'})
         )

netmig.year = netmig.year.astype(int)
netmig.net_migration = netmig.net_migration.astype(int)
netmig.Name = netmig.Name.apply(lambda x: x.replace('-',' '))
netmig.Name = netmig.Name.apply(lambda x: x.replace('St. ','St.'))

In [21]:
print(f'{len(netmig)} and {len(population)}') 


3840 and 3840


In [23]:
netmig.merge(population, on=['Name','year'], how='inner').sort_values('net_migration')

,year,Name,net_migration,population_est
3705,2019,"New York Newark Jersey City, NY NJ PA",-137415,19216182
3321,2018,"New York Newark Jersey City, NY NJ PA",-131299,19276644
2937,2017,"New York Newark Jersey City, NY NJ PA",-109201,19322607
3666,2019,"Los Angeles Long Beach Anaheim, CA",-94938,13214799
2553,2016,"New York Newark Jersey City, NY NJ PA",-91528,19334778
...,...,...,...,...
2774,2017,"Dallas Fort Worth Arlington, TX",87088,7337097
2390,2016,"Dallas Fort Worth Arlington, TX",94103,7194758
2006,2015,"Dallas Fort Worth Arlington, TX",94829,7042566
2079,2015,"Houston The Woodlands Sugar Land, TX",108558,6671808


In [11]:
permits = pd.read_csv('all_permits.csv')
permits.year = permits.year.astype(int)

population = pd.read_csv('cbsa-population.csv', encoding="ISO-8859-1")

permits_population = pd.read_csv('Population_and_Permits_by_metro_(Exact match).csv')
missing_names = permits_population[permits_population.population_est.isna()].Name

In [12]:
missing_names

3                                   Albany, OR
13            Anniston Oxford Jacksonville, AL
17           Atlanta Sandy Springs Roswell, GA
21                       Austin Round Rock, TX
32                            Bend Redmond, OR
                         ...                  
25188    Fayetteville Springdale Rogers, AR MO
25277       Louisville Jefferson County, KY IN
25513                         Coeur dAlene, ID
25556    Fayetteville Springdale Rogers, AR MO
25645       Louisville Jefferson County, KY IN
Name: Name, Length: 2498, dtype: object

In [73]:
from collections import Counter

permits_population = permits.merge(population, on=['Name','year'], how='left')
permits_population.population_est.isna().sum()

2498

In [8]:
#match on state too and fuzzy string match

permits_population[permits_population.population_est.isna()]

,CSA,CBSA,Name,Total,1 Unit,2 Units,3 & 4 Units,5 or more,Number of structures with 5 units or more,Monthly Coverage Percent,month,year,population_est
3,440,10540,"Albany, OR",38,9,0,0,29,2,81,1,2014,NaN
13,999,11500,"Anniston Oxford Jacksonville, AL",7,7,0,0,0,0,62,1,2014,NaN
17,122,12060,"Atlanta Sandy Springs Roswell, GA",1040,1033,2,0,5,1,100,1,2014,NaN
21,999,12420,"Austin Round Rock, TX",1197,720,24,3,450,43,100,1,2014,NaN
32,140,13460,"Bend Redmond, OR",93,88,0,0,5,1,100,1,2014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25188,999,22220,"Fayetteville Springdale Rogers, AR MO",370,333,6,0,31,3,100,9,2019,NaN
25277,350,31140,"Louisville Jefferson County, KY IN",545,255,0,8,282,13,100,9,2019,NaN
25513,518,17660,"Coeur dAlene, ID",181,125,0,3,53,2,91,10,2019,NaN
25556,999,22220,"Fayetteville Springdale Rogers, AR MO",437,389,14,0,34,2,100,10,2019,NaN


In [75]:
permits_population.to_csv('Population_and_Permits_by_metro_(Exact match).csv',index=False)

In [4]:
import censusdata
censusdata.printtable(
    censusdata.censustable('acs1', 2015, 'B23025') 
                    )# Returns information on table B23025 (Employment Status for Population 16+ Years) from the ACS 2015 1-year estimates.


Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B23025_001E  | B23025.  Employment Status for | Total:                                                   | int  
B23025_002E  | B23025.  Employment Status for | In labor force:                                          | int  
B23025_003E  | B23025.  Employment Status for | !! In labor force: Civilian labor force:                 | int  
B23025_004E  | B23025.  Employment Status for | !! !! In labor force: Civilian labor force: Employed     | int  
B23025_005E  | B23025.  Employment Status for | !! !! In labor force: Civilian labor force: Unemployed   | int  
B23025_006E  | B23025.  Employment Status for | !! In labor force: Armed Forces                          | int  
B23025_007E  | B23025.  Employment Status for | Not in labor force                           